In [ ]:
import sys
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean as cmo
import cartopy.crs as ccrs

sys.path.append("..")

from plotfunctions import *

np.seterr(all='ignore')
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
mpl.rcParams['figure.subplot.bottom'] = .02
mpl.rcParams['figure.subplot.top']    = .94
mpl.rcParams['figure.subplot.left']   = .01
mpl.rcParams['figure.subplot.right']  = .9
mpl.rcParams['figure.subplot.wspace'] = .02
mpl.rcParams['figure.subplot.hspace'] = .0

In [ ]:
#Parameters MITgcm
timep= slice("1989-1-1","2018-12-31")


In [ ]:
fig = plt.figure(figsize=(8,5.5))
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=-13)

"""Shean"""
ax = fig.add_subplot(141,projection=proj)
makebackground(ax)

#ds = xr.open_dataset('../../data/davidshean/mos-tile-0_init_masked.nc')
ds = xr.open_dataset('../../data/davidshean/mos-tile-0_masked.nc')
ds = add_lonlat(ds)
melt = ds.Band1

IM = plotmelt(ax,ds.lon,ds.lat,melt)

ax.set_title('Shean et al')

"""Adusumilli"""

ax = fig.add_subplot(142,projection=proj)
makebackground(ax)

ds = xr.open_dataset('../../data/bb0448974g_3_1.h5')
ds = ds.isel(phony_dim_0=slice(3360,4240),phony_dim_1=slice(2120,2480),phony_dim_2=0)
ds = add_lonlat(ds)

m0 = xr.where(np.isnan(ds.w_b),0,ds.w_b)
m1 = xr.where(np.isnan(ds.w_b_interp),0,ds.w_b_interp)

melt = m0
#melt = m0+m1
melt = xr.where(melt>1,melt,1)
melt = xr.where(m0 == 0,np.nan,melt)

IM = plotmelt(ax,ds.lon,ds.lat,melt)

ax.set_title('Adusumilli et al')

"""MITgcm output"""

ax = fig.add_subplot(143,projection=proj)
makebackground(ax)

ds = xr.open_dataset('../../data/paulholland/PAS_851/state2D.nc')
ds = ds.sel(LONGITUDE=slice(245.3,262),LATITUDE=slice(-75.45,-74.05),TIME=timep)
ds = ds.mean(dim='TIME')

lon = ds.LONGITUDE
lat = ds.LATITUDE-.05
melt = -ds.SHIfwFlx * 365.25*86400/920

IM = plotmelt(ax,lon,lat,melt)

ax.set_title('3D Ocean model')

"""Layer"""

ax = fig.add_subplot(144,projection=proj)
makebackground(ax)

for geom in ['Thwaites_e','PineIsland','CrossDots']:
    ds = xr.open_dataset(f'../../results/Layer_{geom}_holland.nc')
    ds = add_lonlat(ds)
    
    melt = np.where(ds.melt>1,ds.melt,1)
    melt = np.where(ds.mask==3,melt,np.nan)
    
    IM = plotmelt(ax,ds.lon,ds.lat,melt)

ax.set_title('2D Layer model')

"""Colorbar"""
ax = fig.add_subplot(144)
ax.set_visible(False)
axins = inset_axes(ax,width="5%",height="100%",loc='lower left',bbox_to_anchor=(1.05, 0., 1, 1),bbox_transform=ax.transAxes,borderpad=0)
cbar = plt.colorbar(IM, cax=axins,extend='both')

cbar.set_ticks([1,10,100])
cbar.set_ticklabels([1,10,100])
#cbar.ax.tick_params(labelsize=21)
cbar.set_label('Melt rate [m/yr]', labelpad=-2)

"""Save figure"""
plt.savefig(f"../../figures/Validation.png",dpi=300)
plt.show()